In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.8 MB/s eta 0:00:00


In [1]:
!pip install SentencePiece

In [3]:
import torch
from tqdm import tqdm

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda', index=0)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 가사 요약 Using T5

In [ ]:
import pandas as pd

file_path1 = '/content/drive/MyDrive/Colab Notebooks/[deep daiv.] 추천시스템 스터디/lyrics_translator/lyrics7_8.xlsx'
file_path2 = '/content/drive/MyDrive/Colab Notebooks/[deep daiv.] 추천시스템 스터디/lyrics_translator/lyrics9_10.xlsx'
lyrics7_8 = pd.read_excel(file_path1)
lyrics9_10 = pd.read_excel(file_path2)

In [15]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')
model.to(device)

def summarize_lyrics(lyrics):
    # Preprocess the text
    inputs = tokenizer.encode("summarize: " + lyrics, return_tensors="pt", max_length=512, truncation=True).to(device)

    # Generate a summary
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4)

    # Decode the summary and return it
    return tokenizer.decode(outputs[0])

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:220: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes 

In [ ]:
# Apply the function to the 'lyrics_english' column of your DataFrame (replace 'df' with your DataFrame's name)
tqdm.pandas()
lyrics7_8['summary'] = lyrics7_8['lyrics_english'].progress_apply(summarize_lyrics)

100%|██████████| 3504/3504 [2:07:09<00:00,  2.18s/it]


In [ ]:
lyrics7_8.to_excel('lyrics7_8_summary.xlsx')

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/[deep daiv.] 추천시스템 스터디/lyrics_translator'

In [ ]:
output_file1 = file_path + '/lyrics7_8_summary.xlsx'

lyrics7_8.to_excel(output_file1)



# 후처리
반복되는 추임새를 중요한 정보로 인식해서 추임새로만 요약하는 결과를 보였음. 따라서 정규식을 이용해서 후처리.

In [ ]:
#요약완료 파일 로드

import pandas as pd

file_path1 = '/content/drive/MyDrive/Colab Notebooks/[deep daiv.] 추천시스템 스터디/lyrics_translator/lyrics_summary.xlsx'
lyrics_summary = pd.read_excel(file_path1, index_col = 0)



In [ ]:
lyrics_summary.head(1)

,name,id,url,Artist,lyrics,lyrics_english,summary
0,Still With You,0eFMbKCRw8KByXyWBw8WO7,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Jung Kook,날 스치는 그대의 옅은 그 목소리\n내 이름을 한 번만 더 불러주세요\n얼어버린 노...,Your pale voice\nPlease call my name once more...,<pad> the moment a day I would have captured m...


In [78]:
#형변환
lyrics_summary['summary'] = lyrics_summary['summary'].astype(str)


## 정규식 패턴 활용 후 이상치 제거 (반영x)

In [76]:
#정규식 패턴 확인용 DF 정의
lyrics_summary_post = lyrics_summary[['url','summary']]

 하단 코드


- (\b\w+(?: \w+)*\b): 이 부분은 단어를 찾는 역할.
\b: 단어의 경계를 나타내는 메타문자.

\w+: 하나 이상의 알파벳 문자 또는 숫자로 구성된 단어를 의미.

(?: \w+)*: 공백과 함께 여러 개의 단어가 반복될 수 있는 그룹입니다. ?:는 비캡처 그룹을 나타내며,
여기서는 반복되는 부분이므로 캡처하지 않음.


- ( \1)+: 이 부분은 반복되는 구조를 찾아냄.

(: 그룹의 시작을 나타냄.

 \1: 앞서 찾은 단어와 동일한 내용을 가진 문자열. \1은 첫 번째 캡처 그룹과 동일한 값을 가리킴.

): 그룹의 끝을 나타냄.

 +: 앞선 그룹이 하나 이상 반복될 수 있음을 의미.

In [72]:
import re

# 정규식 패턴 설정
pattern = r'\b(\w+)\b(?:\s+\1){4,}'


count = 0

# 정규식 패턴 적용하여 해당 행들 출력하기
for index, row in lyrics_summary_post.iterrows():
    summary = row['summary']
    if re.search(pattern, summary):
        print(f"Index: {index}")
        print(f"Summary: {summary}")
        print("-----")
        count += 1

print(f"총 {count}개의 행이 정규식 패턴과 일치.")


Index: 0
Summary: <pad> the moment a day I would have captured more if I knew it would be like this If you face you again I wanted to see the eyes and told you In the ecstatic memory Even if I dance alone, it rains When this fog is lifted, I'll run with a wet feet Catch me then AH AH AH AH Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah</s>
-----
Index: 15
Summary: <pad> if all of the kings had their queens on the throne We would pop champagne and raise a toast To all of the queens who are fighting alone. oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh o</s>
-----
Index: 23
Summary: <pad> don t ya know that I m stronger Don t ya see me in all black Don t ya cry like a baby Ha ha ha ha ha ha ha Who s laughing now Know that it s over Don t ya know that I won t call back Don t ya cry like a baby Ha ha ha ha ha ha Who s laughing now Know that it s over Don t ya know that it s over Don t ya know</s>
-----
In

총 1148개 행이 발견. 해당 행에 대해서 정규식 패턴을 제거하고 재요약을 수행

In [73]:
#새로운 DF생성
df_re_summary = pd.DataFrame(columns=['url', 'lyrics_english_for_pre'])

#원본DF에서 조건에 맞는 행 추출
for index, row in lyrics_summary.iterrows():
    summary = row['summary']
    if re.search(pattern, summary):
        # 조건에 맞는 행을 새로운 데이터프레임에 추가
        df_re_summary = df_re_summary.append({'url': row['url'], 'lyrics_english_for_pre': row['lyrics_english']}, ignore_index=True)


<ipython-input-73-8e1d1dd85412>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_re_summary = df_re_summary.append({'url': row['url'], 'lyrics_english_for_pre': row['lyrics_english']}, ignore_index=True)
<ipython-input-73-8e1d1dd85412>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_re_summary = df_re_summary.append({'url': row['url'], 'lyrics_english_for_pre': row['lyrics_english']}, ignore_index=True)
<ipython-input-73-8e1d1dd85412>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_re_summary = df_re_summary.append({'url': row['url'], 'lyrics_english_for_pre': row['lyrics_english']}, ignore_index=True)
<ipython-input-73-8e1d1dd85412>:9: FutureWarning: The frame.append method is deprecated and will be removed from

## 정규식 패턴을 활용해서 이상치 탐지 후 단어별 분리를 통한 확인

In [83]:
#정규식 패턴 확인용 DF 정의
lyrics_summary_post = lyrics_summary[['url','summary']]

In [84]:
import re

# 정규식 패턴 설정
pattern = r'\b(\w+)\b(?:\s+\1){4,}'


count = 0

# 정규식 패턴 적용하여 해당 행들 출력하기
for index, row in lyrics_summary_post.iterrows():
    summary = row['summary']
    if re.search(pattern, summary):
        print(f"Index: {index}")
        print(f"Summary: {summary}")
        print("-----")
        count += 1

print(f"총 {count}개의 행이 정규식 패턴과 일치.")


Index: 0
Summary: <pad> the moment a day I would have captured more if I knew it would be like this If you face you again I wanted to see the eyes and told you In the ecstatic memory Even if I dance alone, it rains When this fog is lifted, I'll run with a wet feet Catch me then AH AH AH AH Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah Ah</s>
-----
Index: 15
Summary: <pad> if all of the kings had their queens on the throne We would pop champagne and raise a toast To all of the queens who are fighting alone. oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh o</s>
-----
Index: 23
Summary: <pad> don t ya know that I m stronger Don t ya see me in all black Don t ya cry like a baby Ha ha ha ha ha ha ha Who s laughing now Know that it s over Don t ya know that I won t call back Don t ya cry like a baby Ha ha ha ha ha ha Who s laughing now Know that it s over Don t ya know that it s over Don t ya know</s>
-----
In

In [85]:
#새로운 DF생성
df_re_summary = pd.DataFrame(columns=['url', 'lyrics_english_for_pre'])

#원본DF에서 조건에 맞는 행 추출
for index, row in lyrics_summary.iterrows():
    summary = row['summary']
    if re.search(pattern, summary):
        # 조건에 맞는 행을 새로운 데이터프레임에 추가
        df_re_summary = df_re_summary.append({'url': row['url'], 'lyrics_english_for_pre': row['lyrics_english']}, ignore_index=True)


<ipython-input-85-8e1d1dd85412>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_re_summary = df_re_summary.append({'url': row['url'], 'lyrics_english_for_pre': row['lyrics_english']}, ignore_index=True)
<ipython-input-85-8e1d1dd85412>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_re_summary = df_re_summary.append({'url': row['url'], 'lyrics_english_for_pre': row['lyrics_english']}, ignore_index=True)
<ipython-input-85-8e1d1dd85412>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_re_summary = df_re_summary.append({'url': row['url'], 'lyrics_english_for_pre': row['lyrics_english']}, ignore_index=True)
<ipython-input-85-8e1d1dd85412>:9: FutureWarning: The frame.append method is deprecated and will be removed from

In [86]:
from collections import Counter

def remove_repeated_words(text):
    # 단어별로 분리
    words = text.split()

    # 각 단어의 등장 횟수 세기
    word_counts = Counter(words)

    # 5번 이상 등장하는 단어 찾기
    repeated_words = [word for word, count in word_counts.items() if count >= 5]

    # 해당 단어들을 모두 공백 대체
    for word in repeated_words:
        text = re.sub(r'\b' + re.escape(word) + r'\b', ' ', text)

    return text

df_re_summary['lyrics_english_for_pre'] = df_re_summary['lyrics_english_for_pre'].apply(remove_repeated_words)


In [87]:
df_re_summary.tail(60)

,url,lyrics_english_for_pre
1088,https://open.spotify.com/track/4np9LLuQpFsHs51...,ve been thinking bout tomorrow\nInstead of ...
1089,https://open.spotify.com/track/7anzuQWuiCCULZh...,It's fast how time \nAnd \...
1090,https://open.spotify.com/track/75mE23gq3HYRfWI...,Ah \nGotti shiva ah\nYah \...
1091,https://open.spotify.com/track/6JWas9syacxj0Bi...,m so tired being here\nSuppressed by my ...
1092,https://open.spotify.com/track/3OViI6ihMhRlnwy...,You must think stupid\nYou must think ...
1093,https://open.spotify.com/track/0VNJzprpI5sW4oP...,Weep \n D D D \nPlay n skillz\nRun\n...
1094,https://open.spotify.com/track/1hROTTlhrKSiJqb...,heart love\n calling ...
1095,https://open.spotify.com/track/1y8IsEpXxt0uAeU...,Spent 24 hours\n more hours \nYou spent ...
1096,https://open.spotify.com/track/0FNyzQraGkF1Zzf...,What \nDay \n toss turn ...
1097,https://open.spotify.com/track/16GO3QoMy5owKgi...,\n \nMake love and listen music\n...


In [88]:
tqdm.pandas()
df_re_summary['summary'] = df_re_summary['lyrics_english_for_pre'].progress_apply(summarize_lyrics)

100%|██████████| 1148/1148 [40:59<00:00,  2.14s/it]


In [89]:
df_re_summary.to_excel('df_re_summary.xlsx')

#### 데이터 결합

In [94]:
lyrics_summary.set_index('url', inplace=True)
df_re_summary.set_index('url', inplace=True)

lyrics_summary.update(df_re_summary['summary'])

lyrics_summary.reset_index(inplace=True)
df_re_summary.reset_index(inplace=True)


In [10]:
file_path = '/content/drive/MyDrive/Colab Notebooks/[deep daiv.] 추천시스템 스터디/lyrics_translator'

In [17]:
output_file = file_path + '/lyrics_summary_final2.xlsx'

lyrics_summary.to_excel(output_file)
